In [1]:
######################################################
# get python version 
######################################################
from platform import python_version
python_version() 
# print(python_version())
######################################################
# Get GOOGLE API and TOKEN from environmnetal variable
######################################################
import os
CREDENTIALS = os.getenv('GOOGLE_CREDENTIALS')
TOKEN = os.environ.get('GOOGLE_TOKEN')

# write cedentials.json file
File = open("credentials.json", "w")
File.write(CREDENTIALS)
File.close()

# write token.json file
File = open("token.json", "w")
File.write(TOKEN)
File.close()

# print(CREDENTIALS)
# print(TOKEN)

In [2]:
######################################################
# Import libraries
######################################################
import numpy as np
import pandas as pd;
from scipy import integrate
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from IPython.display import display
import ipywidgets as widgets
import IPython
import io
from googleapiclient.http import MediaIoBaseDownload
import shutil
import pathlib
from IPython.display import clear_output
import time
import scipy.io
import ipyspin


######################################################
# Handling some warning messages from matplotlib
######################################################
import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger('matplotlib.font_manager').disabled = True

# make matplotlib to use the Rplot style sheet
plt.style.use('Rplot.mplstyle')
%matplotlib inline
plt.ioff()

# update matplotlib style 
plt.rcParams.update({'font.size': 18})
plt.rcParams['xtick.labelsize'] = 18 
plt.rcParams['ytick.labelsize'] = 18 
plt.rcParams['figure.titlesize'] = 20 

# set the default google drive directory
Default_Project = "1_Farmingham"

In [3]:
######################################################
# Declare all global variables
######################################################
global Gdrive,Creds,Projects_List,Selected_Project,Project_Data_Files,Sensor_Layout_File,Borehole_Properties_File, BaseLine_Data_File, Selected_Data_File, Time_Series_Figure_File
global Boreholes_Sensor_List, Borehole_Properties, Figure, Figure_Axis, Color

######################################################
# Setting some state variables
######################################################
# data file format
Data_File_Ext = ".mat";

# setting color list
Color = (plt.cm.tab20(np.linspace(0,1,16))) # r is 0 to 1 inclusive
Color = ["blue", "orange", "blue", "orange"] # r is 0 to 1 inclusive

# create ./temp directory
Temp_Directory = "./temp"
shutil.rmtree(Temp_Directory, ignore_errors=True)
os.mkdir(Temp_Directory)

In [4]:
######################################################
# extract date time string from Alicia_FileName
# filename = "20220803T200904LDC250_10550MHz_20ns_brillouin_peaks.mat"
# print(extract_date_time(filename))
######################################################
def extract_date_time_string(Alicia_FileName):
    year  = Alicia_FileName[0:4];
    month = Alicia_FileName[4:6];
    day   = Alicia_FileName[6:8];
    hour  = Alicia_FileName[9:11];
    minutes  = Alicia_FileName[11:13];
    seconds  = Alicia_FileName[13:15];
    
    datetime_string = year+"/"+month+"/"+day+"-"+hour+":"+minutes+":"+seconds;
    return datetime_string

######################################################
# Read matlab data file
######################################################
def Read_Mat_Data(FileName):
    f      = scipy.io.loadmat(FileName)
    temp = f["brillouin_peaks"][0]/(1.0*10**6);# 1 degree Celcius = 1.0 MHz
    return temp

In [5]:
######################################################
# Import Google API libraries
######################################################
from __future__ import print_function
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from getfilelistpy import getfilelist

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
SCOPES = ['https://www.googleapis.com/auth/drive']

######################################################
# Connect GOOGLE DRIVE Remotely and save credentials
###################################################### 
def Connect_Google_Drive():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    
    global Gdrive,Creds;
    
    Creds = None
    Gdrive= None;
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        Creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not Creds or not Creds.valid:
        if Creds and Creds.expired and Creds.refresh_token:
            Creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            Creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(Creds.to_json())

    try:
        Gdrive = build('drive', 'v3', credentials=Creds)

    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f'An error occurred: {error}')

    return Gdrive, Creds

######################################################
# Get all the projects in folder "Pile_Load_Test"
# - Save all folder names in Projects_List 
# - Selected_Project is first project i.e. "1_Pratham_Test"
######################################################
def Get_Projects_List():
    
    global Creds,Projects_List,Selected_Project;
    
    #all pile load test files from google drive folder "Geothermal_Test"
    resource   = {"oauth2":Creds,"id":'13oLWXySN0-EJfhSEayL309N4Z_ZFo1Eu',"fields": "files(name)"}
    res        = getfilelist.GetFileList(resource);
    Folders    = res["folderTree"]["names"][1:];   # folder names
    Folders_Id = res["folderTree"]["folders"][1:]; # folder ids
    
    Projects_List = [];
    for i in range(len(Folders)):
        Projects_List.append((Folders[i],Folders_Id[i]));
        
    Selected_Project = Projects_List[2]
        
    return Projects_List

######################################################
# Get all files from Selected_Project folder 
# -saves filenames and their respective ids in variables
######################################################
def Get_Project_Files():
    
    global Gdrive,Selected_Project,Project_Data_Files,Sensor_Layout_File, Time_Series_Figure_File, Borehole_Properties_File, BaseLine_Data_File, Selected_Data_File;
    
    Project_Id = Selected_Project[1]; 
        
    #get all the list of files in the folder
    pageToken = ""
    files=[];
    while pageToken is not None:
        response=Gdrive.files().list(q="'{}' in parents and trashed=false".format(Project_Id), fields= 'nextPageToken, files(id,name)', orderBy="name", pageSize=900, pageToken=pageToken).execute();
        files.extend(response.get('files')); 
        pageToken = response.get('nextPageToken')
        
    Project_Data_Files = [];
    Sensor_Layout_File = None;
    Time_Series_Figure_File = None;
    Borehole_Properties_File = None;
             
    for i in range(len(files)):
        fileName = files[i]["name"];
        fileId   = files[i]["id"];
        file_extension = pathlib.Path(fileName).suffix
                
        if(fileName=="Borehole_Properties.txt"):
            Borehole_Properties_File = (fileName,fileId);
        elif (fileName=="Sensor_Layout.txt"):
            Sensor_Layout_File = (fileName,fileId);
        elif (fileName=="Time_Series_Plot.png"):
            Time_Series_Figure_File = (fileName,fileId);
        elif (file_extension==Data_File_Ext):
            Project_Data_Files.append((extract_date_time_string(fileName)+" PST",fileId));
        else:
            continue;
                    
    BaseLine_Data_File = ("BaseLine.dat",Project_Data_Files[0][1]);
    Selected_Data_File = ("SelectedData.dat",Project_Data_Files[-1][1]);
        
    Download_File(Borehole_Properties_File)
    Download_File(Sensor_Layout_File)
    Download_File(Time_Series_Figure_File)
    Download_File(BaseLine_Data_File)
    Download_File(Selected_Data_File)
   
    return Project_Data_Files, Borehole_Properties_File, Sensor_Layout_File, Time_Series_Figure_File

######################################################
# This function download google drive files given
# the id of the file, and saves it locally 
######################################################
def Download_File(FileName):
    
    global Gdrive;
    
    dirName ="./temp"
    
    filepath= dirName+"/"+FileName[0];
    request = Gdrive.files().get_media(fileId=FileName[1])
    fh = io.BytesIO()
    
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        # print("Download %d%%" % int(status.progress() * 100))
        
    fh.seek(0)
    
    # Write the received data to the file
    with open(filepath, 'wb') as f:
        shutil.copyfileobj(fh, f)

In [6]:
######################################################
# Connect Google Drive and get all the files 
######################################################

Connect_Google_Drive();# connect to Google Drive 
Get_Projects_List();   # get project list 
Get_Project_Files();   # get project files

# print(Projects_List)
# print(Project_Data_Files)

In [7]:
######################################################
# A sensor class for storing the sensor data
######################################################
class Sensor:
    
    def find_nearest(self,array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return idx,array[idx],np.abs(value-array[idx])

    def __init__(self,Name="S",Borehole_Section=[0,1],Cable_Section=[0,1],Gage_Length=1):
        self.Name             = Name;
        self.Borehole_Section = Borehole_Section;
        self.Cable_Section    = Cable_Section;
        self.Borehole_Coordinates= np.array([]);
        self.Cable_Coordinates= np.array([]);
        self.Gage_Length      = Gage_Length;
        self.Data             = np.array([]);
        self.Time             = np.array([]);

    def __str__(self):
        Message = "";
        Message = Message + "Name:              " + str(self.Name) +             "\n";
        Message = Message + "Borehole_Section:  " + str(self.Borehole_Section) + "\n";
        Message = Message + "Cable_Section:     " + str(self.Cable_Section) +    "\n";
        Message = Message + "Borehole_Coordinates: " + str(self.Borehole_Coordinates) +"\n";
        Message = Message + "Cable_Coordinates: " + str(self.Cable_Coordinates) +"\n";
        Message = Message + "Gage_Length:       " + str(self.Gage_Length) +      "\n";
        Message = Message + "Time:              " + str(self.Time) +             "\n";
        Message = Message + "Data:              " + str(self.Data) +             "\n";

        return Message
    
    def Process_Data(self,Data,Time=None):
        Cable_Location         = np.linspace(0,(Data.shape[0]-1)*self.Gage_Length,Data.shape[0]);
        Cable_Start_Index,_,_  = self.find_nearest(Cable_Location,self.Cable_Section[0]);
        Cable_End_Index,_,_    = self.find_nearest(Cable_Location,self.Cable_Section[1]);
        self.Cable_Coordinates = Cable_Location[Cable_Start_Index:Cable_End_Index+1];
        self.Borehole_Coordinates = np.linspace(self.Borehole_Section[0],self.Borehole_Section[1],len(self.Cable_Coordinates));
        Borehole_Section_Length= (self.Borehole_Section[1]-self.Borehole_Section[0]);
        Cable_Section_Length   = (self.Cable_Section[1]-self.Cable_Section[0]);
        if(np.abs(np.round(Borehole_Section_Length)) != np.abs(np.round(Cable_Section_Length))):
            print("\n\nWarning! Sensor " , self.Name, " has Borehole Section Length (m) ", Borehole_Section_Length , " different that Cable Section Length (m) ", Cable_Section_Length , "\n\n")
        self.Time              = Time;
        self.Data              = Data[Cable_Start_Index:Cable_End_Index+1];

# ################ Example ############################
# # note data should be arragend in (location x time)
# location = np.array([0,1,2,3,4]);
# data     = np.array([[2,1], [3,2], [4,4], [5,5], [6,6]])
# # data   = np.array([2,3,4,5,6])
    
# # pile section is r theta z 
# x = Sensor(Name="S",Borehole_Section=[0,2.1],Cable_Section=[1,3], Gage_Length=1);
# SelectedData = Read_Mat_Data("./temp/"+"SelectedData.dat");
# BaseLineData = Read_Mat_Data("./temp/"+"BaseLine.dat");

# # Sensor_Layout_Matrix = np.loadtxt("./temp/Sensor_Layout.txt", comments="#", delimiter=",", dtype=str);
# # print(Sensor_Layout_Matrix)
# print(x)
# x.Process_Data(data)
# print(x)

In [8]:
######################################################
# Get the properties of the borehole 
######################################################
def Get_Borehole_Properties():
    global Borehole_Properties
    Borehole_Properties_Matrix = np.loadtxt("./temp/Borehole_Properties.txt", comments="#", delimiter=",", dtype=str);
    Diameter         = float(Borehole_Properties_Matrix[0,1]);
    Density          = float(Borehole_Properties_Matrix[1,1]);
    Elastic_Modulus  = float(Borehole_Properties_Matrix[2,1]);
    Length           = float(Borehole_Properties_Matrix[3,1]);
    Borehole_Properties = [Diameter, Density, Elastic_Modulus, Length]
    return 

######################################################
# Get Sensor Layout from the "Sensor_Layout.txt" file
######################################################
def Get_Sensor_Layout():
    global Boreholes_Sensor_List;
    file = open("./temp/Sensor_Layout.txt", 'r');
    lines = file.readlines();
    
    Num_Boreholes =1;
    Boreholes_Sensor_List=[];
    gage_length = 1.0;
    
    for index, line in enumerate(lines):
        if(line[0]=="#"):
            continue
        elif(line[0:3]=="GL,"):
            gage_length =float(line[2:].strip().replace(",", "").split(',')[0])    
        elif(line[0]=="-"):
            Borehole_Data = {};
            borehole_name=line.strip().replace("-", "").strip() 
            
            # next four lines is four sensors for the selected borehole
            Sensor_List = np.empty((4),dtype=Sensor);
            for j in range(1,5):
                Sensor_Info  = lines[index+j].strip().split(",");
                name         = Sensor_Info[0].strip();
                borehole_section = np.array([float(Sensor_Info[1]),float(Sensor_Info[2])]);
                cable_section    = np.array([float(Sensor_Info[3]),float(Sensor_Info[4])]);
                Sensor_List[j-1] = Sensor(name,borehole_section, cable_section, gage_length);
                
#                 print(Sensor_List[j-1])
                
            Borehole_Data["Name"]=borehole_name
            Borehole_Data["Sensors"]=Sensor_List
            
            Boreholes_Sensor_List.append(Borehole_Data)
            index = index+5;
            
    return

######################################################
# Process Sensor Data
######################################################
def Process_Sensor_Data():
    global Boreholes_Sensor_List,BaseLine_Data_File, Selected_Data_File;
    
    SelectedData = Read_Mat_Data("./temp/"+Selected_Data_File[0]);
    BaseLineData = Read_Mat_Data("./temp/"+BaseLine_Data_File[0]);
    
    for j in range(len(Boreholes_Sensor_List)):
        Borehole    = Boreholes_Sensor_List[j]["Name"];
        Sensor_List = Boreholes_Sensor_List[j]["Sensors"];

        for i in range(0,len(Sensor_List)):        
            Sensor_List[i].Process_Data(SelectedData-BaseLineData);  

In [9]:
######################################################
# Process Sensor Data for the selected borehole
######################################################
def Plot_Sensor_Data():

    global Boreholes_Sensor_List, Borehole_Properties, Figure_Axis, Figure, Color;
    
    Num_Boreholes=len(Boreholes_Sensor_List);
    columns = 5;
    rows    = int(np.ceil((Num_Boreholes+1)/float(columns)));
    
    ###### Figure Widget #########################
    Figure             = plt.figure(figsize=(24,8*rows));
    Grid_Space         = gridspec.GridSpec(rows, columns, width_ratios=np.ones(columns), height_ratios=np.ones(rows));
    
    Axis = Figure.add_subplot(Grid_Space[0,0]);
    img = mpimg.imread('Illustration.png')
    Axis.imshow(img)
    Axis.axis('off')
    
    row_no   = 0;   column_no= 1;
    handles,labels = [],[];
    for i in range(len(Boreholes_Sensor_List)):
        Borehole_Name = Boreholes_Sensor_List[i]["Name"];
        Sensor_List   = Boreholes_Sensor_List[i]["Sensors"];
        
        if(column_no==columns):
            row_no=row_no+1;
            column_no=0;
            
        Axis = Figure.add_subplot(Grid_Space[row_no,column_no]);
        column_no=column_no+1;
    
        min_z = 0;
        max_z = 0;
          
        for i in range(len(Sensor_List)):
            loc  = Sensor_List[i].Borehole_Coordinates;
            temp = Sensor_List[i].Data;

            min_z = min(loc[0],min_z);
            max_z = max(loc[-1],max_z);
            
            if(Sensor_List[i].Name.split("_")[0]=="Inside"):
                Axis.plot(temp,loc,"--",label=Sensor_List[i].Name,linewidth=2,color=Color[i]);
            else:
                Axis.plot(temp,loc,label=Sensor_List[i].Name,linewidth=2,color=Color[i]);
            
        Axis.set_xlabel("Temp ($^o C$)");Axis.set_ylabel("Depth (m)");Axis.set_ylim([max_z,min_z]);
        Axis.set_title(Borehole_Name+"\n", fontsize = 'small')

        handles, labels = Axis.get_legend_handles_labels();

    Figure.legend(handles, labels, bbox_to_anchor=(0,1.0,1,0.025), loc="upper center",borderaxespad=.5, ncol=12,fontsize = 'small');   
# #     Total_Strain_Axis.legend(bbox_to_anchor=(0,1.02,1,0.2), loc="lower left",mode="expand", borderaxespad=0, ncol=5,fontsize = 'small');

    Figure.tight_layout();
    Figure.savefig("./temp/SensorPlot.png");
    Figure.clf();
    return;

In [10]:
######################################################
# Get pile properties, sensor layout and plot the
# sensor data
######################################################

Get_Borehole_Properties();
Get_Sensor_Layout();
Process_Sensor_Data();
Plot_Sensor_Data();
# plt.show()

In [11]:
################### Spinner Widget ##############
Spinner_Widget = ipyspin.Spinner()
Spinner_Widget.layout.height = '0px'
Spinner_Widget.layout.width = '0px'
Spinner_Widget.lines  = 10  # The number of lines to draw
Spinner_Widget.length = 5  # The length of each line
Spinner_Widget.width  = 5  # The line thickness
Spinner_Widget.radius = .5  # The radius of the inner circle
Spinner_Widget.stop()

In [12]:
###### Projects Widget #########################
Projects_Widget = widgets.Dropdown(
                  options=Projects_List,
                  value=Selected_Project[1],
                  description='<font size="3"> <b> Projects: </b></font>',
                  disabled=False,);

def Projects_Widget_Change(change):
    Spinner_Widget.layout.height = '25px'
    Spinner_Widget.layout.width = '25px'
    Spinner_Widget.spin()
    Update_Label_Widget.value="Updating...... Please Wait";
    
    if change['type'] == 'change' and change['name'] == 'value':
        global Selected_Project,Sensor_Layout_File,Time_Series_Figure_File, Borehole_Properties_File,BaseLine_Data_File,Selected_Data_File;
        
        Selected_Project = (Projects_Widget.label,Projects_Widget.value)
        Get_Project_Files();
        BaseLine_Widget.options = Project_Data_Files;
        SelectedData_Widget.options = Project_Data_Files;
        BaseLine_Widget.value   = Project_Data_Files[0][1];
        SelectedData_Widget.value   = Project_Data_Files[-1][1];  
        Download_File(Borehole_Properties_File);
        Download_File(Sensor_Layout_File);
        Download_File(Time_Series_Figure_File);
        Download_File(BaseLine_Data_File);
        Download_File(Selected_Data_File);
        Get_Borehole_Properties();
        Get_Sensor_Layout();
        Process_Sensor_Data();
        Plot_Sensor_Data();
        SensorPlot_Widget.value=open("./temp/SensorPlot.png", "rb").read();
        Time_Series_SensorPlot_Widget.value=open("./temp/Time_Series_Plot.png", "rb").read();

    Spinner_Widget.stop()
    Update_Label_Widget.value="";

Projects_Widget.observe(Projects_Widget_Change);
###############################################
      
###### BaseLine Widget #########################
BaseLine_Widget = widgets.Dropdown(
                  options=Project_Data_Files,
                  value=Project_Data_Files[0][1],
                  description='<font size="3"> <b> Baseline: </b></font>',
                  disabled=False,);

def BaseLine_Widget_Change(change):
    Spinner_Widget.layout.height = '25px'
    Spinner_Widget.layout.width = '25px'
    Spinner_Widget.spin()
    Update_Label_Widget.value="Updating...... Please Wait";
    
    if change['type'] == 'change' and change['name'] == 'value':
        global BaseLine_Data_File, Time_Series_Figure_File;
        BaseLine_Data_File = ("BaseLine.dat",BaseLine_Widget.value);
        Download_File(BaseLine_Data_File);
        Download_File(Time_Series_Figure_File);
        Process_Sensor_Data();
        Plot_Sensor_Data();
        SensorPlot_Widget.value=open("./temp/SensorPlot.png", "rb").read();
        Time_Series_SensorPlot_Widget.value=open("./temp/Time_Series_Plot.png", "rb").read();
   
    Spinner_Widget.stop()
    Update_Label_Widget.value="";


BaseLine_Widget.observe(BaseLine_Widget_Change);
###############################################

###### DataFile Widget #########################
SelectedData_Widget = widgets.Dropdown(
                options=Project_Data_Files,
                value=Project_Data_Files[-1][1],
                description='<font size="3"> <b> Data: </b></font>',
                disabled=False,);

def SelectedData_Widget_Change(change):
    Spinner_Widget.layout.height = '25px'
    Spinner_Widget.layout.width = '25px'
    Spinner_Widget.spin()
    Update_Label_Widget.value="Updating...... Please Wait";
    
    if change['type'] == 'change' and change['name'] == 'value':
        global Selected_Data_File, Time_Series_Figure_File;
        Selected_Data_File = ("SelectedData.dat",SelectedData_Widget.value);
        Download_File(Selected_Data_File);
        Download_File(Time_Series_Figure_File)
        Process_Sensor_Data();
        Plot_Sensor_Data();
        SensorPlot_Widget.value=open("./temp/SensorPlot.png", "rb").read();
        Time_Series_SensorPlot_Widget.value=open("./temp/Time_Series_Plot.png", "rb").read();
    
    Spinner_Widget.stop()
    Update_Label_Widget.value="";


SelectedData_Widget.observe(SelectedData_Widget_Change);
###############################################

## Plot Sensor Data
Plot_Sensor_Data();

###### Image Widget #########################
SensorPlot_Widget = widgets.Image(
                            value=open("./temp/SensorPlot.png", "rb").read(),
                            format='png',
                            width=20000,
                            height=8000,
                        );
###############################################

###### Image Widget #########################
Time_Series_SensorPlot_Widget = widgets.Image(
                            value=open("./temp/Time_Series_Plot.png", "rb").read(),
                            format='png',
                            width=20000,
                            height=8000,
                        );
###############################################

###### Sync Widget #########################
Synchronize_Widget = widgets.Button(
                    description='Update',
                    disabled=False,
                    button_style='', # 'success', 'info', 'warning', 'danger' or ''
                    tooltip='Synchronize',
                    icon='' # (FontAwesome names without the `fa-` prefix)
                );

def Synchronize_Widget_clicked(button):
#     Spinner_Widget.layout.height = '25px'
#     Spinner_Widget.layout.width = '25px'
#     Spinner_Widget.spin()
#     Get_Project_Files();
#     BaseLine_Widget.options=Project_Data_Files;
#     SelectedData_Widget.options=Project_Data_Files;
#     SelectedData_Widget.value  =Project_Data_Files[-1][1];
#     button.button_style="success";
#     time.sleep(1);
#     button.button_style="";
#     Spinner_Widget.stop()
    
    Spinner_Widget.layout.height = '25px'
    Spinner_Widget.layout.width = '25px'
    Spinner_Widget.spin()
    Update_Label_Widget.value="Updating...... Please Wait";
    
    Selected_Project = (Projects_Widget.label,Projects_Widget.value)
    Get_Project_Files();
    BaseLine_Widget.options     = Project_Data_Files;
    SelectedData_Widget.options = Project_Data_Files;
    BaseLine_Widget.value       = Project_Data_Files[0][1];
    SelectedData_Widget.value   = Project_Data_Files[-1][1];  
    Download_File(Borehole_Properties_File);
    Download_File(Sensor_Layout_File);
    Download_File(Time_Series_Figure_File);
    Download_File(BaseLine_Data_File);
    Download_File(Selected_Data_File);
    Get_Borehole_Properties();
    Get_Sensor_Layout();
    Process_Sensor_Data();
    Plot_Sensor_Data();
    SensorPlot_Widget.value=open("./temp/SensorPlot.png", "rb").read();
    Time_Series_SensorPlot_Widget.value=open("./temp/Time_Series_Plot.png", "rb").read();
        
    Spinner_Widget.stop()
    Update_Label_Widget.value="";
    
Synchronize_Widget.on_click(Synchronize_Widget_clicked)
###############################################

###### Label Widget #############################
Update_Label_Widget = widgets.Label(value="")


In [13]:
# import asyncio
def func2():  
    Spinner_Widget.layout.height = '25px'
    Spinner_Widget.layout.width = '25px'
    Spinner_Widget.spin()
    Update_Label_Widget.value="Updating...... Please Wait";
    
    Selected_Project = (Projects_Widget.label,Projects_Widget.value)
    Get_Project_Files();
    BaseLine_Widget.options     = Project_Data_Files;
    SelectedData_Widget.options = Project_Data_Files;
    BaseLine_Widget.value       = Project_Data_Files[0][1];
    SelectedData_Widget.value   = Project_Data_Files[-1][1];  
    Download_File(Borehole_Properties_File);
    Download_File(Sensor_Layout_File);
    Download_File(Time_Series_Figure_File);
    Download_File(BaseLine_Data_File);
    Download_File(Selected_Data_File);
    Get_Borehole_Properties();
    Get_Sensor_Layout();
    Process_Sensor_Data();
    Plot_Sensor_Data();
    SensorPlot_Widget.value=open("./temp/SensorPlot.png", "rb").read();
    Time_Series_SensorPlot_Widget.value=open("./temp/Time_Series_Plot.png", "rb").read();
        
    Spinner_Widget.stop()
    Update_Label_Widget.value="";
#     await asyncio.sleep(2*60)
    



<center> <font size="6"> <b> Geothermal Analyzer </b></font> <a href="https://github.com/SumeetSinha/Geothermal_Analyzer#readme"  target="_blank">usage</a>
</center> 

In [14]:
display(widgets.HBox((Projects_Widget,BaseLine_Widget,SelectedData_Widget,Spinner_Widget,Synchronize_Widget,Update_Label_Widget)))
display(SensorPlot_Widget)
display(Time_Series_SensorPlot_Widget)
# func2()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x06\xc0\x00\x00\x02@\x08\x06\x00\x00\x003F\xa3\xb5\x…

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x06@\x00\x00\x06\xd6\x08\x06\x00\x00\x00\xea3{\xf9\x…